# Batch Scoring

This script does batch scoring.
1. It parses, transforms data in GCS to be scored, 
2. Loads the model in GCS 
(note: you need to update the modelVersion variable with the version number from the model training notebook),
3. Uses the model to predict
4. Persists predictions to BigQuery

Copyright 2022 Google LLC

 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.

In [ ]:
spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml import PipelineModel
import sys
from datetime import datetime
import random

In [ ]:
# 1a. Arguments
pipelineID = random.randint(1, 10000)
projectNbr = "YOUR_PROJECT_NBR"
projectID = "YOUR_PROJECT_ID"
modelVersion = pipelineID
displayPrintStatements = True

In [ ]:
# 1b. Variables 
appBaseName = "customer-churn-model"
appNameSuffix = "batch-scoring"
appName = f"{appBaseName}-{appNameSuffix}"
modelBaseNm = appBaseName
bqDatasetNm = f"{projectID}.customer_churn_ds"
modelBucketUri = f"gs://s8s_model_bucket-{projectNbr}/{modelBaseNm}/hyperparameter-tuning/{modelVersion}"
scoreDatasetBucketFQN = f"gs://s8s_data_bucket-{projectNbr}/customer_churn_score_data.csv"
bigQueryOutputTableFQN = f"{bqDatasetNm}.batch_predictions"
scratchBucketUri = f"s8s-spark-bucket-{projectNbr}/{appBaseName}/pipelineId-{pipelineID}/{appNameSuffix}/"
pipelineExecutionDt = datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
# 1c. Display input and output
if displayPrintStatements:
    print("Starting batch_scoring for Customer Churn Predictions")
    print(".....................................................")
    print(f"The datetime now is - {pipelineExecutionDt}")
    print(" ")
    print("INPUT-")
    print(f"....pipelineID={pipelineID}")
    print(f"....modelVersion={modelVersion}")
    print(f"....projectNbr={projectNbr}")
    print(f"....projectID={projectID}")
    print(f"....displayPrintStatements={displayPrintStatements}")
    print(" ")
    print("OUTPUT-")
    print(f"....BigQuery Table={bigQueryOutputTableFQN}")
    print(f"SELECT * FROM {bigQueryOutputTableFQN} WHERE model_version='{modelVersion}' AND pipeline_id='{pipelineID}' AND pipeline_execution_dt='{pipelineExecutionDt}' LIMIT 10" )
    
    

In [ ]:
# 2. Spark Session creation
print('....Initializing spark & spark configs')
spark = SparkSession.builder.appName(appName).getOrCreate()

# Spark configuration setting for writes to BigQuery
spark.conf.set("parentProject", projectID)
spark.conf.set("temporaryGcsBucket", scratchBucketUri)

# Add Python modules
sc.addPyFile(f"gs://s8s_code_bucket-{projectNbr}/pyspark/common_utils.py")
import common_utils

In [ ]:
# 3. Read data to be scored from GCS
print('....Read batch scoring input and profile')
scoreRawDF = spark.read.options(inferSchema = True, header= True).csv(scoreDatasetBucketFQN)
if displayPrintStatements:
    print(scoreRawDF.count())

In [ ]:
# 4. Display data, display summary stats
if displayPrintStatements:
    scoreRawDF.show(2)
    scoreRawDF.describe().show()

In [ ]:
# 5. Replace spaces, space with null values in the TotalCharges and MonthlyCharges columns
print('....Data pre-process: fnReplaceSpaceWithNone in TotalCharges and MonthlyCharges')
spaceReplacedDF = common_utils.fnReplaceSpaceWithNone(scoreRawDF)
if displayPrintStatements:
    print(spaceReplacedDF.count())

In [ ]:
# 6. Replace non-numeric values in the TotalCharges and MonthlyCharges columns
print('....Data pre-process: ReplaceNotANumberWithNone in TotalCharges and MonthlyCharges')
nanReplacedDF = common_utils.fnReplaceNotANumberWithNone(spaceReplacedDF)
if displayPrintStatements:
    print(nanReplacedDF.count())

In [ ]:
# 7. Drop rows with null in columns
print('....Data pre-process: Drop rows with none')
nullDroppedDF = nanReplacedDF.na.drop()

if displayPrintStatements:
    print(nullDroppedDF.count())

In [ ]:
# 8. Replace 'No internet service' across columns to 'No'
print('....Data pre-process: Replace -No internet service- across columns with -No-')
partiallyProcessedDF = common_utils.fnReplaceWithNoForInternetService(nullDroppedDF)
if displayPrintStatements:
    print(partiallyProcessedDF.count())

In [ ]:
# 9. Add a bin/bucket category for tenure range using Spark SQL and write transformed to dataframe
print('....Data pre-process: Create bins by tenure months') 
scoreTargetDF = common_utils.fnAddBinForTenure(partiallyProcessedDF, True, spark)
if displayPrintStatements:
    print(scoreTargetDF.count())
    scoreTargetDF.show(2)                            

In [ ]:
# 10. Format dataframe names for column name format consistency
scorableDF = scoreTargetDF.select("customerID", "gender", "SeniorCitizen", "Partner", "Dependents", "tenure", "Tenure_Group", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "MonthlyCharges", "TotalCharges") \
                                .toDF("customer_id", "gender", "senior_citizen", "partner", "dependents", "tenure", "tenure_group", "phone_service", "multiple_lines", "internet_service", "online_security", "online_backup", "device_protection", "tech_support", "streaming_tv", "streaming_movies", "contract", "paperless_billing", "payment_method", "monthly_charges", "total_charges") 

if displayPrintStatements:
    print(scorableDF.count())
    scorableDF.show(2)

In [ ]:
# 11. Load the pre-trained, persisted model in GCS
print('....Scoring: Load model out of GCS into memory') 
model = PipelineModel.load(f"{modelBucketUri}/bestModel/")

In [ ]:
# 12. Batch scoring
print('....Scoring: Execute model.transform') 
batchScoreResultsDF = model.transform(scorableDF) \
                           .withColumn("model_version", lit(modelVersion).cast("string")) \
                           .withColumn("pipeline_id", lit(pipelineID).cast("string")) \
                           .withColumn("pipeline_execution_dt", lit(pipelineExecutionDt)) 

if displayPrintStatements:
    batchScoreResultsDF.show(2)

In [ ]:
# 13. Persist to BigQuery
print('....Persisting: Batch scoring results to BigQuery')
batchScoreResultsDF.select("customer_id", "gender", "senior_citizen", "partner", "dependents", "tenure", "tenure_group", "phone_service", "multiple_lines", "internet_service", "online_security", "online_backup", "device_protection", "tech_support", "streaming_tv", "streaming_movies", "contract", "paperless_billing", "payment_method", "monthly_charges", "total_charges","prediction","model_version","pipeline_id","pipeline_execution_dt") \
.write.format('bigquery') \
.mode("append")\
.option('table', bigQueryOutputTableFQN) \
.save()
